# クロージャとnonlocal
自分が定義された時のスコープを覚えている関数

## 0. 関数の中で定義する関数
関数の中で関数を定義することが可能。

In [2]:
def double(x, y):
    def add(x, y):
        return x + y

print(double(1, 2))

None


#### 関数の中で関数を定義して、一体何に使うのか？
長くて読みにくい処理に名前をつけて、短く表情したい時に関数内に関数を定義。  
そこでさらに疑問が湧く。処理に名前を付けたいなら外側で定義したら？関数内に定義するメリットは2つ。
- 外側の関数のローカル変数にも内側の関数からアクセス可能。
- 内側の関数名が外から見えず、他から使われないことが保証できるので読む時に楽。

その結果、ある関数内でのみ必要な処理を気軽にコンパクトに書ける。

## 1. クロージャ
自由変数を持った関数  
（Pyhtonには明確な言語仕様が存在しない。。）

### 1.1. 普通の関数
例えば、関数fは自身が定義されたグローバルスコープを覚えている。xを参照しても0を返してくれる。

In [3]:
x = 0

# 関数fは自由変数xを持つクロージャ
def f():
    return x

print(f())

0


> 関数または関数の中で定義された関数に自由変数がある場合、定義される各def文またはlambda式はクロージャを作成する。

[PEP 227 -- Statically Nested Scopes](https://www.python.org/dev/peps/pep-0227/)

> プログラミングテストにおいては、自由変数とは関数の中で参照される局所変数や引数以外の変数を意味する。

[自由変数と束縛変数 - Wikipedia](https://ja.wikipedia.org/wiki/%E8%87%AA%E7%94%B1%E5%A4%89%E6%95%B0%E3%81%A8%E6%9D%9F%E7%B8%9B%E5%A4%89%E6%95%B0)

### 1.2. 関数の中で定義された関数
以下の例では関数gは自分が定義されたスコープ（関数fのローカルスコープ）を覚えているので、グローバル変数y=0ではなく、y=1を出力する。gは自由変数yを持つクロージャ。

In [4]:
y = 0
def f():
    y = 1
    # 関数gは自由変数yを持つクロージャ
    def g():
        return y
    return g

h = f()
print(h())

1


> nested score   
> （ネストされたスコープ）外側で定義されている変数を参照する機能。  

[nested scope - 用語集](https://docs.python.org/ja/3/glossary.html#term-nested-scope)


## 2. クロージャの使いどき
こんな機能どこで使うのか？ここでは関数の中で定義された関数がnetedスコープを使う例について紹介する。

### 2.1. デコレータ
引数を取るデコレータを定義したい時は、nestedスコープが欲しい。例えば実行時間を計測する以下のデコレータ。

In [12]:
import time

def measure(func):
    # 関数 wrapper は自由変数 func を持つクロージャ
    def wrapper(*args, **kargs):
        start_time = time.perf_counter()
        result = func(*args, **kargs)
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        print(f'{func.__name__}: {execution_time}')
        return result

    return wrapper

# 素因数分解
@measure
def factorize(n):
    b = 2
    fct = []
    while b * b <= n:
        while n % b == 0:
            n /= b
            fct.append(b)
        b = b + 1
    if n > 1:
        fct.append(int(n))
    return fct

print(factorize(99))

factorize: 8.392999916395638e-06
[3, 3, 11]


> 私はPythonを関数型言語として見ていないが、クロージャの導入は価値があったと考えている。クロージャは他の多くのプログラミング機能の拡張の開発の役に立っているからである。例えば、新スタイルクラスやデコレータなど、いくつかの新しい機能はクロージャを利用している。（訳注：クロージャは元々関数型言語を参考にして導入された機能だそう。次に見る、カリー化と部分適用も関数型言語でのテクニックだと聞いている。）

[Pythonの"関数型"の機能の起源 - The History of Python.js](http://python-history-jp.blogspot.com/2009/05/python.html)


### 2.2. カリー化と部分適用
カリー化、部分適用をしたい時は、nested スコープが欲しい。  
(カリー化はプログラミング上のテクニックというよりは、理論的な面での扱いやすさ、という側面が大きい。あらゆる関数を1引数の関数として統一的に扱えるので、理論的に考える上では扱いが楽になる）

In [18]:
# 通常
# def floordiv(a, b):
#     return a // b

# step 1. カリー化
def splitter(b):
    # 関数 split は自由変数 b　を持つクロージャ
    def split(a):
        return a // b
    return split

# step 2. 部分適用
# floordivの変数の1つbを固定し、任意の数aを半分にする関数halfを作成
half = splitter(2)

print(half(4))

2


因みにカリー化したり部分適用することが面倒だなと思ったら、標準ライブラリ functools の中にあるpartial 関数を使うと、既にある関数を元にカリー化することなく部分適用が可能

## 3. nonlocal 文
クロージャが導入されたことでnestedスコープの変数を参照できるようになった。しかし、そのままでは変更することが不可能。そこでPython3.0からnonlocal文が導入された。

### 3.1 カウンタ

In [19]:
def create_counter():
    c = 0
    def count():
        nonlocal c   # 1つ外側の変数 c を束縛する
        c = c + 1
        return c
    return count

count = create_counter()
print(count())
print(count())
print(count())
print(count())
print(count())
print(count())

1
2
3
4
5
6


> nonlocal文は、列挙された識別子がブローバルを除く一つ外側のスコープえ先に束縛された変数を参照するようにする。

[7.13. nonlocal 文](https://docs.python.org/ja/3/reference/simple_stmts.html#the-nonlocal-statement)

### 3.2. 擬似乱数
擬似乱数を生成する関数

In [21]:
def linear_congruential(a, x, b, m):
    def _():
        nonlocal x
        x = (a * x + b) % m
        return x
    return _

random = linear_congruential(48271, 8, 0, 2**31-1)
print(random())
print(random())
print(random())
print(random())
print(random())

386168
1460846352
1741224500
285379567
1596966799


## 4. nonlocal文の使いどき
専用のオブジェクトを作らないで、変数だけで簡単に状態を持たせたい時だと思っている。   
（通常は何か状態を持たせる時はクラスを作成する。）

In [23]:
# nonlocalを使用しない場合
class Counter:
    def __init__(self):
        self._times = 0
    def count(self):
        self._times += 1
        return self._times

# nonlocalを使用した場合
def create_counter():
    c = 0
    def count():
        nonlocal c   # 1つ外側の変数 c を束縛する
        c = c + 1
        return c
    return count

nonlocalを使えば、わざわざクラスを定義する必要がない。上記の例では行数が変わらなかったが、一般的なケースではnonlocalを使う方がコードが短くなる。が、多くの場合は分かりやすさからクラス定義文を使う方が望ましい。また、nonlocal文はあまり見かけない。

## 5. 2種類の名前の探し方

In [24]:
x = 0

def f():
    print(x)

def g():
    x = 1
    f()

g()

0


### 5.1. 静的スコープ
変数を「定義」した箇所から外側に向けて広がるスコープ。  
Pythonは静的スコープを採用している。

### 5.2 動的スコープ
変数を「参照」した箇所から外側に向けて広がるスコープ。  
Pythonは動的スコープを採用していない。割と危険で、最近の言語ではほとんど採用されていない。

## 6. クラスのスコープ
クラスにもスコープがあり、少し独特な動作をする。クラス定義文で生成されるスコープは名前解決の際に飛ばされる。

一般的な場合、

In [28]:
x = 0

def g():
    x = 1   # <-- こっちが参照
    def f():
        return x  # 1
    return f

f = g()
f()

1

クラススコープの場合、クラス定義文で生成されるスコープはスキップ。

In [31]:
x = 0  # <-- こっちが参照
class C:
    x = 1  # 名前解決時にはスキップ
    def f(self):
        print(x)

obj = C()
obj.f()

0


self をつければクラス定義文のスコープにある変数、クラス変数を参照できる。

In [35]:
x = 0
class C:
    x = 1
    def f(self):
        print(self.x)

obj = C()
obj.f()

1


### 6.2 3つの例外
Pythonの名前解決は、そのほかにも2つの例外がある。

> 議論   
> ... 名前解決のルールは、静的スコープの言語に典型的なもの。ただし、3つの例外がある。   
> - クラススコープの名前は、参照できない。
> - global文は通常のルールをショートカットします。
> - 変数は宣言されない

[Statically Nested Scopes - PEP 227](https://www.python.org/dev/peps/pep-0227/#discussion)

補足：  
「global文は通常のルールをショートカットする」というのは、global文が付与された変数はnestedスコープをショートカットして直接globalスコープを参照することを指している。   
「変数は宣言されない」というのは、例えばローカル変数を使う場合は、JavaScriptのようにlet、constと書かなくてよいという意味。